# Import necessary Libraries

In [ ]:
from bs4 import BeautifulSoup
from collections import namedtuple
from redmine import Redmine
from redmine.exceptions import ServerError
from redmine.exceptions import ForbiddenError
from redmine.exceptions import ImpersonateError

import csv
import datetime
import html2text
import io
import logging
import os
import requests
import sys
import traceback

# Project Settings

In [ ]:
# Trac Settings
trac_url = "https://trac/"
trac_project = ""

trac_user = ""
trac_password = ""

# Redmine Settings
redmine_api_url = "https://redmine/"
redmine_project_id = 'dipf'

redmine_key = ""

In [ ]:
import logging
logger = logging.getLogger()
report_logger = logging.getLogger()
log_formatter = logging.Formatter(
        fmt='%(asctime)s - %(levelname)s: %(message)s',
        datefmt="%Y-%m-%d %H:%M:%S"
    )
report_formatter = logging.Formatter(
        fmt='%(message)s',
        datefmt="%Y-%m-%d %H:%M:%S"
    )
stdout_hanlder = logging.StreamHandler(sys.stdout)
stdout_hanlder.setFormatter(log_formatter)
logger.addHandler(stdout_hanlder)
file_handler = logging.FileHandler(
    'import.log',
    mode='w',
    encoding='utf-8')
file_handler.setFormatter(log_formatter)
logger.addHandler(file_handler)
report_file_handler = logging.FileHandler(
    'import_report.log',
    mode='w',
    encoding='utf-8')
# Set Basic Log-Level for this
logger.setLevel(logging.INFO)

In [ ]:
h = html2text.HTML2Text()
h.ignore_links = False

In [ ]:
logger.info('Logging all Action')

# Mappings

In [ ]:
trac_data = dict()

In [ ]:
payload = {
    'format': 'csv',
    'max': 10000,
    'col': ['id', 
            'type',
            'summary',
            'status',
            'solution',
            'reporter',
            'owner',
            'cc',
            'priority',
            'milestone',
            'component',
            'resolution',
            'time',
            'changetime',
            'estimatedhours',
            'billable',
            'totalhours',
            'keywords',
           ],
    'order': 'id',
}

In [ ]:
logger.info('Process Base Data')
base_data_request = requests.get(trac_url + trac_project + '/query', auth=(trac_user, trac_password), params=payload)
if base_data_request.status_code == 200 and base_data_request.headers['content-type'] == 'text/csv;charset=utf-8':
    csv_data = io.StringIO(base_data_request.text)
    csv_reader = csv.DictReader(csv_data)
    for row in csv_reader:
        ticket_data = dict()
        ticket_data['id'] = row['id']
        logger.debug('Process Ticket: %s', ticket_data['id'])
        ticket_data['type'] = row['type']
        ticket_data['title'] = row['summary']
        ticket_data['status-solution'] = row['status'] + '-' + row['resolution']
        ticket_data['reporter'] = row['reporter']
        ticket_data['assigned_to'] = row['owner']
        ticket_data['watchers'] = row['cc']
        ticket_data['priority'] = row['priority']
        ticket_data['version'] = row['milestone']
        ticket_data['category'] = row['component']
        if row['time']:
            ticket_data['start_date'] = datetime.datetime.strptime(row['time'][0:19], '%Y-%m-%d %H:%M:%S').date()
        if row['changetime']:
            ticket_data['due_date'] = datetime.datetime.strptime(row['changetime'][0:19], '%Y-%m-%d %H:%M:%S').date()
        ticket_data['estimatedhours'] = row['estimatedhours']
        ticket_data['billable'] = row['billable']
        ticket_data['totalhours'] = row['totalhours']
        ticket_data['keywords'] = row['keywords']
        
        trac_data[ticket_data['id']] = ticket_data

In [ ]:
logger.info('Process Advanced Data (Attachements, Changelog, Time Booking)')
for index in trac_data.keys():
    logger.debug('Process Ticket: %s', ticket_data['id'])
    ticket_data = trac_data[index]
    r = requests.get(trac_url + trac_project + '/ticket/' + str(index), auth=(trac_user, trac_password), params={'format': 'csv'})
    if r.status_code == 200 and r.headers['content-type'] == 'text/csv;charset=utf-8':
        csv_data = io.StringIO(r.text)
        csv_reader = csv.DictReader(csv_data)
        for row in csv_reader:
            ticket_data['description_raw'] = row['description']
    r = requests.get(trac_url + trac_project + '/ticket/' + str(index), auth=(trac_user, trac_password))
    if r.status_code == 200 and r.headers['content-type'] == 'text/html;charset=utf-8':
        soup = BeautifulSoup(r.text, 'html.parser')
        # Get Description
        desc_part = soup.find(id='ticket').find('div', {'class': 'description'}).find('div', {'class': 'searchable'})
        ticket_data['description_html'] = str(desc_part)
        ticket_data['description_markdown'] = h.handle(str(desc_part))
        # Get Attachements
        attachements = soup.find('div' , {'id': 'attachments'}).find_all('dt')
        ticket_data['attachments'] = []
        for attachment in attachements:
            try:
                os.stat('transfer-files/' + str(index))
            except:
                os.mkdir('transfer-files/' + str(index))
            title = attachment.a.text
            file = attachment.find('a', {'class': 'trac-rawlink'})
            with open('transfer-files/' + index + '/' + title, 'wb') as handle:
                file_r = requests.get(trac_url + file.get('href'), auth=(trac_user, trac_password))
                if file_r.ok:
                    for chunk in file_r.iter_content(chunk_size=1024):
                        if chunk:
                            handle.write(chunk)
            ticket_data['attachments'].append(Attachement(filename=title, path='transfer-files/' + index + '/' + title))
        # Changelog --> Comments and hours
        if soup.find(id='changelog'):
            changes = soup.find(id='changelog').find_all('div', {'class': 'change'})
            ticket_data['changes'] = []
            for change in changes:
                change_id = 0
                change_id_elem = change.find('span', {'class': 'cnum'})
                if change_id_elem:
                    change_id = int(change_id_elem.attrs['id'][8:])
                raw_timestamp = change.find('a', {'class': 'timeline'})
                user = raw_timestamp.next_sibling[8:].strip()
                raw_timestamp = raw_timestamp.get('title')[0:10]
                timestamp = datetime.datetime.strptime(raw_timestamp, '%Y-%m-%d').date()
                changelog = change.find('ul', {'class': 'changes'})
                ttype = 'comment'
                if changelog:
                    for elem in changelog.find_all('li'):
                        if elem.strong.text == "Add Hours to Ticket":
                            if len(elem.find_all('em')) == 2:
                                hours = float(elem.find_all('em')[1].text)
                                ttype = 'TimeEntry'
                comment_raw = change.find('div', {'class': 'comment'})
                comment_html = str(comment_raw)
                comment_markdown = str(h.handle(str(comment_raw))).strip()
                if ttype == 'TimeEntry':
                    ticket_data['changes'].append({'type': 'time_entry',
                                                   'trac_change_id': change_id,
                                                   'user': user,
                                                   'hours': hours,
                                                   'timestamp': timestamp,
                                                   'comment': comment_markdown})
                else:
                    ticket_data['changes'].append({'type': 'comment_entry',
                                                   'trac_change_id': change_id,
                                                   'user': user,
                                                   'timestamp': timestamp,
                                                   'comment': comment_markdown})
        
        

In [ ]:
import json
with open('data.json', 'w') as file:
    file.write(str(trac_data))

# Start writing into Redmine

In [ ]:
error_list = []
for index in trac_data:
    ticket_data = trac_data[index]
    if ticket_data.get('redmine_id'):
        del ticket_data['redmine_id']

In [ ]:
MapData = namedtuple('MapData', ['assigned_to', 'version', 'ccs', 'category', 'attachments'])
def map_data(ticket_data):
    assigned_to = None
    if ticket_data['assigned_to']:
        assigned_to = users.get(ticket_data['assigned_to']).id
    logger.debug('Assigned To: %s', assigned_to)

    version = None
    if ticket_data['version'] is not None:
        version_name = ticket_data['version'].strip()
        version_mapping = milestones.get(version_name)
        if version_mapping:
            version = version_mapping.id
        else:
            version = ''
            print('Could not find version: "' + version + '"')
    logger.debug('Version: %s', version)

    ccs = set()
    cc = ticket_data.get('watchers').split(',')
    for user in cc:
        if '@' in user:
            user = user[:-2]
        user = user.strip()
        if users.get(user):
            ccs.add(users.get(user).id)
    ccs = list(ccs)
    logger.debug('Watchers: %s', ', '.join([str(cc) for cc in ccs]))

    category = None
    if ticket_data['category']:
        category = categories.get(ticket_data['category']).id
    logger.debug('Category: %s', category)

    attachments = []
    if ticket_data['attachments']:
        attachments = [{'path': elem.path, 'filename': elem.filename} for elem in ticket_data['attachments']]  
    logger.debug('Attachments: %s', ', '.join([at.get('filename') for at in attachments])) 
    
    return MapData(assigned_to=assigned_to, version=version, ccs=ccs, category=category, attachments=attachments)

In [ ]:
logger.info('Prevent multiple Creation of tickets, by linking both')
for index in trac_data:
    ticket_data = trac_data[index]
    if ticket_data.get('redmine_id'):
        del ticket_data['redmine_id']


redmine = Redmine(
    redmine_api_url,
    key=redmine_key
)
issues = redmine.issue.filter(project_id=redmine_project_id, status_id='*')
for issue in issues:
    # try to not duplicate import Issues
    trac_id = issue.custom_fields[0].value
    if trac_id:
        trac_data[trac_id]['redmine_id'] = issue.id

In [ ]:
logger.info('Write Ticket Data to Redmine')
for index in trac_data.keys():
    ticket_data = trac_data[index]
    user = ticket_data['reporter']
    logger.debug('try to impersonate as %s --> %s', user, users.get(user).login)
    redmine = Redmine(
        redmine_api_url,
        key=redmine_key,
        impersonate=users.get(user).login
    )
    
    if ticket_data.get('redmine_id'):
        logger.debug('Ticket : %s already exists.', index)
    else:
        logger.info('Processing Ticket %s', index)
        
        assigned_to, version, ccs, category, attachments = map_data(ticket_data)
        
        try:
            issue = redmine.issue.create(
                project_id=redmine_project_id,
                subject=ticket_data['title'],
                tracker_id=trackers.get(ticket_data['type']).id,
                description=ticket_data['description_markdown'],
                status_id=status_solutions.get(ticket_data['status-solution']).id,
                priority_id=priorities.get(ticket_data['priority']).id,
                fixed_version_id=version,
                category_id=category,
                assigned_to_id=assigned_to,
                watcher_user_ids=ccs,
                start_data=ticket_data['start_date'],
                #due_date=ticket_data['due_date'],
                estimated_hours=ticket_data['estimatedhours'],
                custom_fields=[
                    {'id': 1, 'value': int(ticket_data['id'])}
                ],
                uploads=attachments
            )
            ticket_data['redmine_id'] = issue.id
        except Exception as e:
            logger.error(e)
            error_list.append({'type': 'not created',
                               'ticket': 'https://dev.starzel.de/dipf/ticket/' + index,
                               'index': index,
                              })

In [ ]:
repeat_error_list = dict()

In [ ]:
for elem in error_list:
    index = elem['index']
    issue = None
    ticket_data = trac_data[index]
    user = ticket_data['reporter']
    redmine = Redmine(
        redmine_api_url,
        key=redmine_key,
        impersonate=users.get(user).login
    )
    
    if ticket_data.get('redmine_id'):
        logger.debug('Ticket : %s already exists.', index)
        issue = redmine.issue.get(ticket_data.get('redmine_id'))
    else:
        logger.info('Processing Ticket %s', index)
        
        assigned_to, version, ccs, category, attachments = map_data(ticket_data)
        
        try:
            issue = redmine.issue.create(
                project_id=redmine_project_id,
                subject=ticket_data['title'],
                tracker_id=trackers.get(ticket_data['type']).id,
                status_id=status_solutions.get(ticket_data['status-solution']).id,
                priority_id=priorities.get(ticket_data['priority']).id,
                fixed_version_id=version,
                category_id=category,
                assigned_to_id=assigned_to,
                start_data=ticket_data['start_date'],
                watcher_user_ids=ccs,
                custom_fields=[
                    {'id': 1, 'value': int(ticket_data['id'])}
                ],
                estimated_hours=ticket_data['estimatedhours'],
                uploads=attachments
            )
            ticket_data['redmine_id'] = issue.id
            
        except Exception as e:
            logger.error(e)
            repeat_error_list[index] = {
                'type': 'not created',
                'ticket': 'https://dev.starzel.de/dipf/ticket/' + index,
                'index': index,
            }
            continue
    try:
        issue.description=ticket_data['description_raw']
        issue.save()
    except Exception as e:
        logger.error(e)
        repeat_error_list[index] = {
                'type': 'decription_not_written',
                'ticket': 'https://dev.starzel.de/dipf/ticket/' + index,
                'index': index,
                'redmine_ticket_id': issue.id
            }
    
        

In [ ]:
not_created_error = []
description_error = []
for key, entry in repeat_error_list.items():
    if entry['type'] == 'not created':
        not_created_error.append('* ' + str(entry['ticket']))
    elif entry['type'] == 'decription_not_written':
        description_error.append('* #' + str(entry['redmine_ticket_id']) + ' ' + str(entry['ticket']))
    
    
print('## Tickets die nicht angelegt werden konnten:\n\n'+ '\n'.join(not_created_error))
print('\n\n## Ticket bei denen die Beschreibung einen Fehler verursacht hat:\n\n'+ '\n'.join(description_error))

In [ ]:
issues_without_description = set()

redmine = Redmine(
    redmine_api_url,
    key=redmine_key,
)

issues = redmine.issue.filter(project_id=redmine_project_id, status_id='*')
for issue in issues:
    if not hasattr(issue, 'description'):
        trac_id = issue.custom_fields[0].value
        issues_without_description.add((issue.id, trac_id))

print('Die Tickets wurden ohne Beschreibung angelegt.\n\nSumme: ' + str(len(issues_without_description)))
for elem in issues_without_description:
    print('* #' + str(elem[0]) + '  https://dev.starzel.de/dipf/ticket/'+ str(elem[1]))

# Second run - import Changelog

In [ ]:
second_run_error_list = []

In [ ]:
logger.info('Write Changelog Data to Redmine')
for index in trac_data.keys():
    logger.info('Processing Ticket %s', index)
    ticket_data = trac_data[index]
    if ticket_data.get('redmine_id'):
        changes = ticket_data['changes']
        redmine = Redmine(
            redmine_api_url,
            key=redmine_key,
        )
        redmine_issue = redmine.issue.get(ticket_data['redmine_id'], include="journals")
        for elem in redmine_issue.journals:
            logger.debug('Journal Entry %s already exists', elem.id)
        
        for change in changes:
            logger.debug('Change: \n%s', change)
            comment = change['comment'].strip()
            if len(comment) > 1024:
                second_run_error_list.append({'type': 'comment_to_long', 'ticket':'https://dev.starzel.de/dipf/ticket/' + index + '#' + str(change.get('trac_change_id'))})
                continue
            if change.get('hours'):
                hours=change.get('hours')
                if hours < 0.0:
                    second_run_error_list.append({'type': 'negative_hours', 'ticket':'https://dev.starzel.de/dipf/ticket/' + index + '#' + str(change.get('trac_change_id')), 'hours': hours})
                    hours = 0.0
            if change['type'] == 'time_entry' and change.get('time_entry_id') is None:
                user = change['user']
                if user in users.keys():
                    try:
                        redmine = Redmine(
                            redmine_api_url,
                            key=redmine_key,
                            impersonate=user
                        )
                        time_entry = redmine.time_entry.create(
                            issue_id=ticket_data['redmine_id'],
                            hours=hours,
                            spent_on=change.get('timestamp'),
                            activity_id=9, # Development
                            comments=comment)
                        change['time_entry_id'] = time_entry.id
                    except ImpersonateError as e:
                        redmine = Redmine(
                            redmine_api_url,
                            key=redmine_key,
                        )
                        time_entry = redmine.time_entry.create(
                            issue_id=ticket_data['redmine_id'],
                            hours=hours,
                            spent_on=change.get('timestamp'),
                            activity_id=9, # Development
                            comments=comment)
                        change['time_entry_id'] = time_entry.id
            elif change['type'] == 'comment_entry' and change.get('journal_entry_id') is None:
                user = change['user']
                if user in users.keys() and comment:
                    note=user + ':\n\n' + comment
                    try:
                        redmine = Redmine(
                            redmine_api_url,
                            key=redmine_key,
                            impersonate=user
                        )
                        if redmine.issue.update(ticket_data['redmine_id'], notes=note):
                            issue = redmine.issue.get(ticket_data['redmine_id'], include='journal')
                            journal_id = issue.journals[issue.journals.total_count -1].id
                            change['journal_entry_id'] = journal_id
                    except ImpersonateError:
                        redmine = Redmine(
                            redmine_api_url,
                            key=redmine_key,
                        )
                        if redmine.issue.update(ticket_data['redmine_id'], notes=note):
                            issue = redmine.issue.get(ticket_data['redmine_id'], include='journal')
                            journal_id = issue.journals[issue.journals.total_count -1].id
                            change['journal_entry_id'] = journal_id
                    except:
                        second_run_error_list.append({'type': 'unkown', 'ticket':'https://dev.starzel.de/dipf/ticket/' + index + '#' + str(change.get('trac_change_id'))})
                        
                    
            else:
                logger.info('Change alreadey exists: https://dev.starzel.de/dipf/ticket/' + str(index) + '#' + str(change.get('trac_change_id')))
                


In [ ]:
from pprint import pprint
#pprint(second_run_error_list)

comment_error = []
time_error = []
unknow_error = []
for entry in second_run_error_list:
    if entry['type'] == 'negative_hours':
        time_error.append('* ' + str(entry['ticket']) + ' - hours: ' + str(entry['hours']) )
    elif entry['type'] == 'comment_to_long':
        comment_error.append('* ' + entry['ticket'])
    elif entry['type'] == 'unkown':
        unknow_error.append('* ' + entry['ticket'])
    
    
print('### Negative Zeitbuchung:\n\n'+ '\n'.join(time_error))
print('\n\n### Kommentar zu lang:\n\n'+ '\n'.join(comment_error))
print('\n\n### Unbekannter Fehler:\n\n'+ '\n'.join(unknow_error))

# Verify import

In [ ]:
redmine = Redmine(
    redmine_api_url,
    key=redmine_key,
)
issues = redmine.issue.filter(project_id=redmine_project_id, status_id='*')

verify_errors = []
for issue in issues:
    trac_id = issue.custom_fields[0].value
    if trac_id != '':
        trac_ticket_data = trac_data[trac_id]
        if issue.id != trac_ticket_data['redmine_id']:
            verify_errors.append('Issue: ' + str(issue.id) + ' mapping not correct')
        if issue.subject != trac_ticket_data['titel']:
            verify_errors.append('Issue: ' + str(issue.id) + ' Subject not correct')
        if issue.description != trac_ticket_data['description_markdown']:
            verify_errors.append('Issue: ' + str(issue.id) + ' Description not correct')
print(verify_errors)